In [1]:
import pandas as pd
import sqlite3


#proof of concept - only has data from 9/7/19-9/21-19
conn = sqlite3.connect('mta_scrape1.db')
query = "SELECT *, DATE(DATETIME) AS DATE from entries"

df = pd.read_sql_query(query,conn)

df.head()



,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS,DATE
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323,2019-09-14
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325,2019-09-14
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354,2019-09-14
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428,2019-09-14
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483,2019-09-14


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615308 entries, 0 to 615307
Data columns (total 13 columns):
id          615308 non-null int64
CA          615308 non-null object
UNIT        615308 non-null object
SCP         615308 non-null object
STATION     615308 non-null object
LINENAME    615308 non-null object
DIVISION    615308 non-null object
DATETIME    615308 non-null object
TIME        615308 non-null object
DESC        615308 non-null object
ENTRIES     615308 non-null int64
EXITS       615308 non-null int64
DATE        615308 non-null object
dtypes: int64(3), object(10)
memory usage: 61.0+ MB


In [3]:
#Test for see if series works
df['ENTRIES'].head()

0    7198818
1    7198834
2    7198847
3    7198929
4    7199125
Name: ENTRIES, dtype: int64

In [4]:
#create turnstile index column
df['TURNSTILE_INDEX'] = df['CA'] + '-' +  df['UNIT'] + '-' + df['SCP'] + '-' + df['STATION']

df.head()

,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS,DATE,TURNSTILE_INDEX
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323,2019-09-14,A002-R051-02-00-00-59 ST
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325,2019-09-14,A002-R051-02-00-00-59 ST
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354,2019-09-14,A002-R051-02-00-00-59 ST
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428,2019-09-14,A002-R051-02-00-00-59 ST
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483,2019-09-14,A002-R051-02-00-00-59 ST


In [5]:
#create summary of # of unique turnstiles/station
n_turnstiles = df.groupby('STATION')['TURNSTILE_INDEX'].nunique().sort_values(ascending=False)
                    
#top 10 stations by number of turnstiles    
n_turnstiles.head(10)

STATION
34 ST-PENN STA     100
FULTON ST           96
23 ST               73
GRD CNTRL-42 ST     69
86 ST               63
CANAL ST            58
34 ST-HERALD SQ     54
CHAMBERS ST         54
59 ST               52
14 ST               51
Name: TURNSTILE_INDEX, dtype: int64

In [6]:
# calculate minimum entry number by station, day and turnstile
flow_min = df.groupby(['STATION','DATE','TURNSTILE_INDEX'])['ENTRIES'].min().reset_index()
flow_min.columns = ['STATION','DATE','TURNSTILE_INDEX','Min']

flow_min.head()


,STATION,DATE,TURNSTILE_INDEX,Min
0,1 AV,2019-08-31,H007-R248-00-00-00-1 AV,14944966
1,1 AV,2019-08-31,H007-R248-00-00-01-1 AV,60535823
2,1 AV,2019-08-31,H007-R248-00-03-00-1 AV,370717463
3,1 AV,2019-08-31,H007-R248-00-03-01-1 AV,2493370
4,1 AV,2019-08-31,H007-R248-00-03-02-1 AV,6456464


In [7]:
# calculate maximum entry number by station, day and turnstile
flow_max  = df.groupby(['STATION','DATE','TURNSTILE_INDEX'])['ENTRIES'].max().reset_index()
flow_max.columns = ['STATION','DATE','TURNSTILE_INDEX','Max']

flow_max.head()

,STATION,DATE,TURNSTILE_INDEX,Max
0,1 AV,2019-08-31,H007-R248-00-00-00-1 AV,14944966
1,1 AV,2019-08-31,H007-R248-00-00-01-1 AV,60535823
2,1 AV,2019-08-31,H007-R248-00-03-00-1 AV,370717463
3,1 AV,2019-08-31,H007-R248-00-03-01-1 AV,2493370
4,1 AV,2019-08-31,H007-R248-00-03-02-1 AV,6456464


In [8]:
#join min and max dataframes together. Pandas recognizes common columns and uses as index
flow_join = flow_min.merge(flow_max)

#calculate difference to get flow numbers
flow_join['diff'] = flow_join['Max'] - flow_join['Min']

flow_join.head(10)

,STATION,DATE,TURNSTILE_INDEX,Min,Max,diff
0,1 AV,2019-08-31,H007-R248-00-00-00-1 AV,14944966,14944966,0
1,1 AV,2019-08-31,H007-R248-00-00-01-1 AV,60535823,60535823,0
2,1 AV,2019-08-31,H007-R248-00-03-00-1 AV,370717463,370717463,0
3,1 AV,2019-08-31,H007-R248-00-03-01-1 AV,2493370,2493370,0
4,1 AV,2019-08-31,H007-R248-00-03-02-1 AV,6456464,6456464,0
5,1 AV,2019-08-31,H008-R248-01-00-00-1 AV,290375,290511,136
6,1 AV,2019-08-31,H008-R248-01-00-01-1 AV,2771897,2772258,361
7,1 AV,2019-08-31,H008-R248-01-00-02-1 AV,687259,687959,700
8,1 AV,2019-08-31,H008-R248-01-00-03-1 AV,955609012,955610126,1114
9,1 AV,2019-08-31,H008-R248-01-00-04-1 AV,594932,596632,1700


In [9]:
#summarize the above dataframe down to station by day values

daily_station_totals = flow_join.groupby(['STATION','DATE'])['diff'].sum().reset_index()
daily_station_totals.columns = ['STATION','DATE','Sum']

daily_station_totals.head(15)

,STATION,DATE,Sum
0,1 AV,2019-08-31,4011
1,1 AV,2019-09-01,3298
2,1 AV,2019-09-02,2862
3,1 AV,2019-09-03,13779
4,1 AV,2019-09-04,14587
5,1 AV,2019-09-05,16582
6,1 AV,2019-09-06,17110
7,1 AV,2019-09-07,5085
8,1 AV,2019-09-08,4241
9,1 AV,2019-09-09,15961
